# MCS 275 Spring 2024 Homework 13 Solutions

* Course Instructor: Emily Dumas


### Web application downloads

It might be easier to just get a version of the web application that has the solution to this homework incorporated already.  Here are links to ZIPs of those.

* [taskgain-1.2.zip](https://www.dumas.io/teaching/2024/spring/mcs275/data/taskgain-1.2.zip)
* [activetask-1.2.zip](https://www.dumas.io/teaching/2024/spring/mcs275/data/activetask-1.2.zip)

## 2. Troubled task audit

In the task tracking application, let's define two types of "troubled" tasks:

1. A **stale task** is one that is in the "waiting" state and where the creation time is at least one week ago.

1. A **long-running task** is one that is the "in progress" state and where the last update time is at least one week ago.

The idea behind these definitions is that both types of tasks might require additional attention or action---stale tasks have been waiting for a long time and may have been forgotten, while long-running tasks may have hit some kind of obstacle preventing progress.

Add a feature to the task tracking web app so that accessing the URL `localhost:5000/audit/troubled/` will show a page with the application logo/header, followed by a list of stale tasks, followed by a list of long-running tasks.  Each list should have a header (h2) above it indicating what it shows.  For each task in either list, the description, task id, state, and owner should be shown.  The new page should use the same CSS stylesheet as the rest of the application.

The modifications should consist of
* One function added to `taskgain.py` or `activetask.py`
* One HTML template added to the `templates/` subdirectory, which I recommend calling `troubled_task_audit.html`.

**Test your work** but once it is working, do not submit the entire application.  Instead, copy and paste just the function you added to the python script into a new file called `hwk13.py`, and put the required header at the top of that file.  Then submit two files:`hwk13.py` and your new HTML template `troubled_task_audit.html`.

### Solution

The template we show is specific to ActiveTask, but the changes for TaskGain are straightforward.

#### Function

In [ ]:
# Needs to be added to activetask.py or taskgain.py after `app` is defined
@app.route("/audit/troubled/")
def troubled_task_audit():
    "Show troubled tasks"
    now = time.time()
    week_ago = now - 7 * 24 * 60 * 60
    con = sqlite3.connect(DB_FN)

    stale_results = con.execute(
        """
        SELECT taskid, description, owner, status, shared, updated_ts
        FROM tasks
        WHERE status = ? AND created_ts <= ?
        ORDER BY created_ts DESC;
        """,
        [ST_WAIT, week_ago],
    )
    stale_tasks = []
    for row in stale_results:
        stale_tasks.append(
            {
                "taskid": row[0],
                "description": row[1],
                "owner": row[2],
                "status": row[3],
                "status_str": STATUS_DESC[row[3]],
                "shared_code": row[4],
                "shared_str": SHARED_DESC[row[4]],
                "updated_ts": row[5],
                "updated_str": timefmt.ts_fmt(row[5]),
                "updated_delta_str": timefmt.tsdiff_fmt(now - row[5]),
            }
        )

    lrt_results = con.execute(
        """
        SELECT taskid, description, owner, status, shared, updated_ts
        FROM tasks
        WHERE status = ? AND updated_ts <= ?
        ORDER BY created_ts DESC;
        """,
        [ST_PROGRESS, week_ago],
    )
    lrt_tasks = []
    for row in lrt_results:
        lrt_tasks.append(
            {
                "taskid": row[0],
                "description": row[1],
                "owner": row[2],
                "status": row[3],
                "status_str": STATUS_DESC[row[3]],
                "shared_code": row[4],
                "shared_str": SHARED_DESC[row[4]],
                "updated_ts": row[5],
                "updated_str": timefmt.ts_fmt(row[5]),
                "updated_delta_str": timefmt.tsdiff_fmt(now - row[5]),
            }
        )
    return render_template(
        "troubled_task_audit.html",
        stale_tasks=stale_tasks,
        lrt_tasks=lrt_tasks,
    )


#### Template

See [troubled_task_audit.html](https://github.com/daviddumas/mcs275spring2024/blob/hw13soln/samplecode/web/activetask/templates/troubled_task_audit.html) on github.

### Revision history

* 2024-04-18 Initial publication